## Descargando mapa y data:

In [1]:
import geopandas as gpd

link="https://github.com/chorrillos/spatial/raw/main/worldMap.geojson"
WorldMap=gpd.read_file(link)
WorldMap.columns

Index(['FIPS', 'ISO2', 'ISO3', 'UN', 'NAME', 'AREA', 'POP2005', 'REGION',
       'SUBREGION', 'LON', 'LAT', 'geometry'],
      dtype='object')

In [2]:
import pandas as pd
link2="https://github.com/xmarquez/democracyData/raw/master/data-raw/EIU%20Democracy%20Index%202020.csv"
datademo=pd.read_csv(link2)
datademo.columns

Index(['country', 'eiu', 'year'], dtype='object')

In [3]:
# data 2021
#install bs4 y html5lib
link3="https://en.wikipedia.org/wiki/Democracy_Index"
data2021=pd.read_html(link3,flavor='bs4',header=0)[6]
data2021.columns

Index(['Rank',
       '.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Δ Rank',
       'Country', 'Regime type', 'Overall score', 'Δ Score',
       'Elec­toral pro­cessand plura­lism', 'Func­tioningof govern­ment',
       'Poli­ticalpartici­pation', 'Poli­ticalcul­ture', 'Civilliber­ties'],
      dtype='object')

Transformar de estilo largo a ancho

In [ ]:
datademo=datademo.pivot(index='country', columns='year', values='eiu')
datademo.reset_index(inplace=True)
datademo

In [ ]:
#note
datademo.columns

In [ ]:
datademo.columns=[str(y) for y in datademo.columns]
datademo.columns

In [ ]:
datademo.info()

Limpiar 2021

In [ ]:
data2021.columns

In [ ]:
data2021.drop(data2021.columns[[0,1,3,4,5]],axis=1,inplace=True)

In [ ]:
#nota caracteres
data2021.columns.to_list()

In [ ]:
#mejor
data2021.columns.str.replace('\N{SOFT HYPHEN}|\s', '',regex=True).str.lower().to_list()

In [ ]:
data2021.columns=data2021.columns.str.replace('\N{SOFT HYPHEN}|\s', '',regex=True).str.lower().to_list()
data2021.columns

In [ ]:
#test
datademo.merge(data2021).shape

Detectar qué falta

In [ ]:
sinmatchDATA=list(set(datademo.country)- set(data2021.country))
sinmatch2021=list(set(data2021.country)-set(datademo.country) )
len(sinmatchDATA), len(sinmatch2021)

In [ ]:
from thefuzz import process
[(process.extractOne(country,sinmatch2021),country) for country in sorted(sinmatchDATA)]

In [ ]:
#test
data2021[data2021.country.str.contains("Kyr")]

In [ ]:
#cambios
data2021.loc[data2021.country=="Republic of the Congo","country"]="Congo (Brazzaville)"
#data2021.loc[data2021.country=="Democratic Republic of the Congo","country"]="Democratic Republic of Congo"
data2021.loc[data2021.country=="United Arab Emirates","country"]="UAE"
data2021.loc[data2021.country=="United Kingdom","country"]="UK"
data2021.loc[data2021.country=="United States","country"]="US"
data2021.loc[data2021.country=="Kyrgyzstan","country"]="Kyrgyz Republic"

In [ ]:
# dict de cambios:
sinmatchDATA=list(set(datademo.country)- set(data2021.country))
sinmatch2021=list(set(data2021.country)-set(datademo.country) )

cambios2021Name={process.extractOne(country,sinmatch2021)[0]:country for country in sorted(sinmatchDATA)}
cambios2021Name

In [ ]:
# ejecutando cambios 
data2021.country.replace(cambios2021Name,inplace=True)
data2021

In [ ]:
datademo.merge(data2021).shape

In [ ]:
datademo=datademo.merge(data2021)

Para mapa:

In [ ]:
sinmatchDATA=list(set(datademo.country)- set(WorldMap.NAME))
sinmatchMAP=list(set(WorldMap.NAME)-set(datademo.country) )
len(sinmatchDATA), len(sinmatchMAP)

In [ ]:
from thefuzz import process
[(process.extractOne(country,sinmatchMAP),country) for country in sorted(sinmatchDATA)]

## Corrigiendo texto clave

In [ ]:
WorldMap.loc[WorldMap.NAME=="Kyrgyzstan","NAME"]="Kyrgyz Republic"
WorldMap.loc[WorldMap.NAME=="Korea, Democratic People's Republic of","NAME"]="North Korea"
WorldMap.loc[WorldMap.NAME=="Korea, Republic of","NAME"]="South Korea"
WorldMap.loc[WorldMap.NAME=="Burma","NAME"]="Myanmar"
WorldMap.loc[WorldMap.NAME=="United Arab Emirates","NAME"]="UAE"
WorldMap.loc[WorldMap.NAME=="United Kingdom","NAME"]="UK"
WorldMap.loc[WorldMap.NAME=="United States","NAME"]="US"

In [ ]:
sinmatchDATA=list(set(datademo.country)- set(WorldMap.NAME))
sinmatchMAP=list(set(WorldMap.NAME)-set(datademo.country) )
len(sinmatchDATA), len(sinmatchMAP)

In [ ]:
[(process.extractOne(country,sinmatchMAP),country) for country in sorted(sinmatchDATA)]

In [ ]:
# dict de cambios:
cambiosMapaName={process.extractOne(country,sinmatchMAP)[0]:country for country in sorted(sinmatchDATA)}
cambiosMapaName

In [ ]:
# ejecutando cambios 
WorldMap.NAME.replace(cambiosMapaName,inplace=True)

## Preparando mapa

In [ ]:
# merge hacia MAPA
WorldMap=WorldMap.merge(datademo, left_on='NAME', right_on="country")

In [ ]:
WorldMap.shape

In [ ]:
WorldMap.info()

In [ ]:
WorldMap.iloc[:,27:]=WorldMap.iloc[:,27:].astype('float')
WorldMap.describe()

In [ ]:
# guardando data
WorldMap.to_file("WorldMapDemo.gpkg", layer='DISTRITO', driver="GPKG")